In [97]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

As I found that I had an issue with get_shared_words - function can return the same word from text1 multiple times even though in text2 it appears only once (cause I didn't deleted it from both texts once match found)

This notebook is execution of clean algo with fixed function, in order to be able to compare if there is main difference in results

In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder

import sys
import imp

sys.path.append('../src/')
import data.reader as dataReader
import utils.utils as thesisUtils
import similarities.cosine as thesisCosineSimilarity
import vocabulary.vocabulary as thesisVocabulary
import features.model_features as thesisModelFeatures
import features.factory as thesisFactoryFeatures
import data.corpus_stats as thesisCorpusStats

imp.reload(dataReader)
imp.reload(thesisUtils)
imp.reload(thesisVocabulary)
imp.reload(thesisCorpusStats)
imp.reload(thesisModelFeatures)
imp.reload(thesisFactoryFeatures)
imp.reload(thesisCosineSimilarity)

<module 'similarities.cosine' from '../src/similarities/cosine.py'>

In [71]:
london_corpus = dataReader.CorpusByNewLine.london()
zwickau_corpus = dataReader.CorpusByNewLine.zwickau()

In [112]:
np.average([ len("".join(i.split())) for i in london_corpus.corpus])

506.3659305993691

In [85]:
featuresFactory  = thesisFactoryFeatures.FeaturesFactory(
    london_corpus = london_corpus,
    zwickau_corpus = zwickau_corpus
)

In [4]:
london_zwickau_similarities = thesisCosineSimilarity.CrossVersionSimilarity5Gram(london_corpus, zwickau_corpus)
# london_zwickau_similarities.calculate()

zwickau_london_similarities = thesisCosineSimilarity.CrossVersionSimilarity5Gram(zwickau_corpus, london_corpus)
# zwickau_london_similarities.calculate()

In [5]:
# london_zwickau_similarities.save()
# zwickau_london_similarities.save()
london_zwickau_similarities.load()
zwickau_london_similarities.load()

In [6]:
burchard_corpus_by_london = dataReader.BurchardCorpus(london_corpus, zwickau_corpus)
burchard_corpus_by_zwickau = dataReader.BurchardCorpus(zwickau_corpus, london_corpus)

In [7]:
london_leftovers = dataReader.LeftoversCorpus(london_corpus, zwickau_corpus)
zwickau_leftovers = dataReader.LeftoversCorpus(zwickau_corpus, london_corpus)

In [8]:
print(len(london_leftovers.corpus_for_predictions()))
print(len(zwickau_leftovers.corpus_for_predictions()))

157
146


In [9]:
def filter_short_p(corpus):
    return list(filter(lambda x: len(x.split()) > 20, corpus))

In [10]:
def build_burchard_corpus_with_predictions(burchard_corpus, wrong_predictions_by_london, wrong_predictions_by_zwickau):
    is_burchard = True
    temp_corpus = [ [p, is_burchard, is_burchard] for p in burchard_corpus ]
    
    for prediction in wrong_predictions_by_london:
        temp_corpus[prediction.index][1] = False
    for prediction in wrong_predictions_by_zwickau:
        temp_corpus[prediction.index][2] = False

    return temp_corpus

In [11]:
london_leftofvers_long = filter_short_p(london_leftovers.corpus)
zwickau_leftofvers_long = filter_short_p(zwickau_leftovers.corpus)
burchard_corpus_by_london_corpus_long = filter_short_p(burchard_corpus_by_london.corpus)
burchard_corpus_by_zwickau_corpus_long = filter_short_p(burchard_corpus_by_zwickau.corpus)

In [17]:
print(len(burchard_corpus_by_london_corpus_long))
print(len(burchard_corpus_by_zwickau_corpus_long))

203
203


In [12]:
burchard_by_london_VS_zwickau_features_df = thesisModelFeatures.create_features_df(
    None,
    zwickau_leftofvers_long,
    burchard_corpus_by_london_corpus_long,
    n_gram = (2,5),
    features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
)
burchard_by_zwickau_VS_zwickau_features_df = thesisModelFeatures.create_features_df(
    None,
    zwickau_leftofvers_long,
    burchard_corpus_by_zwickau_corpus_long,
    n_gram = (2,5),
    features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
)
burchard_by_london_VS_london_features_df = thesisModelFeatures.create_features_df(
    london_leftofvers_long,
    None,
    burchard_corpus_by_london_corpus_long,
    n_gram = (2,5),
    features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
)
burchard_by_zwickau_VS_london_features_df = thesisModelFeatures.create_features_df(
    london_leftofvers_long,
    None,
    burchard_corpus_by_zwickau_corpus_long,
    n_gram = (2,5),
    features = { 'tfidf', 'inner_mean_cosine_similarity_score' }
)

n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram
n_gram_feature_name: 2_5_gram


## london VS zwickau

In [86]:
london_vs_zwickau_features_df = featuresFactory.london_VS_zwickau(n_gram = 5)

n_gram_feature_name: 5_gram
n_gram_feature_name: 5_gram


In [91]:
london_vs_zwickau_modesl_experiment = thesisModelFeatures.ModelsExperiment(london_vs_zwickau_features_df, 'london_vs_zwickau')

In [92]:
london_vs_zwickau_modesl_experiment.run_cross_validate()

running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.067 minutes
running: GaussianProcessClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Elapsed time to compute the cross validate: 0.395 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.080 minutes
running: MLPClassifier
Elapsed time to compute the cross validate: 7.165 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.042 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.118 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.294 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 1.645 minutes


In [93]:
london_vs_zwickau_modesl_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBClassifier,0.644888,0.641815,0.641103,0.643763,0.642272,0.643763
DecisionTreeClassifier,0.638522,0.633810,0.626616,0.633978,0.627473,0.633978
RandomForestClassifier,0.564386,0.562292,0.559406,0.564946,0.561047,0.564946
AdaBoostClassifier,0.563087,0.561845,0.556595,0.561828,0.557048,0.561828
KNeighborsClassifier,0.510956,0.503810,0.482487,0.508280,0.485266,0.508280
GaussianNB,0.463535,0.466131,0.419225,0.474946,0.425272,0.474946
GaussianProcessClassifier,0.347066,0.458690,0.357748,0.472258,0.366915,0.472258
MLPClassifier,0.438806,0.434643,0.418199,0.438925,0.421507,0.438925


In [94]:
london_vs_zwickau_modesl_experiment.run_greed_search_cv_for_n_best_models(3)

running: DecisionTreeClassifier
Elapsed time to compute the greed search cv: 0.516 minutes
running: RandomForestClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.5316129  0.49516129 0.51526882 0.53537634 0.52892473 0.52225806
 0.52892473 0.53215054 0.5316129  0.49516129 0.51526882 0.53537634
 0.52892473 0.52225806 0.52892473 0.53215054 0.48548387 0.5116129
 0.48526882 0.51505376 0.5183871  0.5183871  0.52806452 0.51172043
 0.55150538 0.52494624 0.52817204 0.53505376 0.52526882 0.54182796
 0.54505376 0.55172043 0.55150538 0.52494624 0.52817204 0.53505376
 0.52526882 0.54182796 0.54505376 0.55172043 0.47215054 0.4855914
 0.46591398 0.48903226 0.50204301 0.50172043 0.51505376 0.51827957
 0.5116129  0.54795699 0.55462366 0.55139785 0.5516129  0.54806452
 0.5383871  0.54505376 0.5116129  0.54795699 0.55462366 0.55139785
 0.5516129  0.54806452 0.5383871  0.54505376 0.50505376 0.4916129
 0.47193548 0.48182796 0.50473118 0.49172043 0.49516129 0.50827957
 0.5311828  0.5544086  0.55763441 0

Elapsed time to compute the greed search cv: 7.607 minutes
running: XGBClassifier
Elapsed time to compute the greed search cv: 35.071 minutes


In [95]:
for i in london_vs_zwickau_modesl_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

0.660752688172043
DecisionTreeClassifier(max_depth=6)
0.5838709677419355
RandomForestClassifier(max_depth=11, n_estimators=250, random_state=0)
0.6770967741935484
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.4, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=5,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [96]:
london_vs_zwickau_modesl_experiment.greed_search_cv_results[2].best_params_

{'gamma': 0.4, 'max_depth': 3, 'min_child_weight': 5}

## burchard VS zwickau

In [13]:
burchard_by_london_VS_zwickau_models_experiment = thesisModelFeatures.ModelsExperiment(burchard_by_london_VS_zwickau_features_df)

In [14]:
burchard_by_london_VS_zwickau_models_experiment.run_cross_validate()

running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.157 minutes
running: GaussianProcessClassifier
Elapsed time to compute the cross validate: 0.885 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.119 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.071 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.076 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.658 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 2.041 minutes


In [15]:
burchard_by_london_VS_zwickau_models_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBClassifier,0.833020,0.808810,0.814038,0.825546,0.821450,0.825546
AdaBoostClassifier,0.783148,0.772262,0.773547,0.782521,0.780704,0.782521
RandomForestClassifier,0.740203,0.702976,0.702336,0.731092,0.717339,0.731092
DecisionTreeClassifier,0.693169,0.688214,0.687327,0.699496,0.696937,0.699496
GaussianProcessClassifier,0.792409,0.634167,0.608110,0.690756,0.637669,0.690756
KNeighborsClassifier,0.642882,0.550833,0.490248,0.616134,0.532242,0.616134
GaussianNB,0.602404,0.520476,0.457501,0.584454,0.499864,0.584454


In [16]:
burchard_by_london_VS_zwickau_models_experiment.run_greed_search_cv_for_n_best_models(3)

running: RandomForestClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.71092437 0.71663866 0.72243697 0.72529412 0.72243697 0.7310084
 0.72815126 0.73386555 0.71092437 0.71663866 0.72243697 0.72529412
 0.72243697 0.7310084  0.72815126 0.73386555 0.58747899 0.58168067
 0.58168067 0.58168067 0.58168067 0.58168067 0.58168067 0.58168067
 0.69092437 0.73092437 0.72815126 0.72815126 0.71957983 0.72529412
 0.73386555 0.73386555 0.69092437 0.73092437 0.72815126 0.72815126
 0.71957983 0.72529412 0.73386555 0.73386555 0.60764706 0.59033613
 0.58168067 0.58453782 0.58168067 0.58168067 0.58168067 0.58168067
 0.6994958  0.72529412 0.70529412 0.7310084  0.7310084  0.73672269
 0.73957983 0.73672269 0.6994958  0.72529412 0.70529412 0.7310084
 0.7310084  0.73672269 0.73957983 0.73672269 0.60747899 0.60168067
 0.59882353 0.59596639 0.59882353 0.59310924 0.58739496 0.58739496
 0.68235294 0.71672269 0.71092437 

Elapsed time to compute the greed search cv: 13.180 minutes
running: AdaBoostClassifier
Elapsed time to compute the greed search cv: 58.082 minutes
running: XGBClassifier
Elapsed time to compute the greed search cv: 59.349 minutes


In [23]:
for i in burchard_by_london_VS_zwickau_models_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

0.7510924369747899
RandomForestClassifier(max_depth=9, n_estimators=400, random_state=0)
0.856890756302521
AdaBoostClassifier(learning_rate=1, n_estimators=2000)
0.8399159663865545
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.4, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=5,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [24]:
burchard_by_zwickau_VS_zwickau_models_experiment = thesisModelFeatures.ModelsExperiment(burchard_by_zwickau_VS_zwickau_features_df)

In [25]:
burchard_by_zwickau_VS_zwickau_models_experiment.run_cross_validate()

running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.138 minutes
running: GaussianProcessClassifier
Elapsed time to compute the cross validate: 0.872 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.118 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.068 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.070 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.652 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 1.918 minutes


In [26]:
burchard_by_zwickau_VS_zwickau_models_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBClassifier,0.830416,0.813095,0.817459,0.825546,0.823580,0.825546
AdaBoostClassifier,0.793551,0.779524,0.782360,0.791008,0.789328,0.791008
DecisionTreeClassifier,0.726609,0.719167,0.717216,0.724958,0.723798,0.724958
RandomForestClassifier,0.740095,0.685714,0.687324,0.716639,0.702652,0.716639
GaussianProcessClassifier,0.790594,0.624881,0.595643,0.682185,0.626282,0.682185
KNeighborsClassifier,0.648487,0.550714,0.491053,0.616134,0.532939,0.616134
GaussianNB,0.642063,0.532381,0.474211,0.595882,0.514955,0.595882


In [27]:
burchard_by_zwickau_VS_zwickau_models_experiment.run_greed_search_cv_for_n_best_models(3)

running: DecisionTreeClassifier
Elapsed time to compute the greed search cv: 0.921 minutes
running: AdaBoostClassifier
Elapsed time to compute the greed search cv: 53.156 minutes
running: XGBClassifier
Elapsed time to compute the greed search cv: 55.258 minutes


In [28]:
for i in burchard_by_zwickau_VS_zwickau_models_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

0.7652941176470588
DecisionTreeClassifier(criterion='entropy', max_depth=90)
0.8253781512605041
AdaBoostClassifier(learning_rate=1, n_estimators=500)
0.8427731092436973
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=3,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [40]:
burchard_by_zwickau_VS_zwickau_models_experiment.greed_search_cv_results[2].best_params_

{'gamma': 0.0, 'max_depth': 5, 'min_child_weight': 3}

## burchard vs london

In [29]:
burchard_by_london_VS_london_models_experiment = thesisModelFeatures.ModelsExperiment(burchard_by_london_VS_london_features_df)

In [30]:
burchard_by_london_VS_london_models_experiment.run_cross_validate()

running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.156 minutes
running: GaussianProcessClassifier
Elapsed time to compute the cross validate: 0.967 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.121 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.073 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.077 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.709 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 2.071 minutes


In [31]:
burchard_by_london_VS_london_models_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
XGBClassifier,0.757065,0.729881,0.728194,0.747222,0.737307,0.747222
RandomForestClassifier,0.769375,0.703661,0.696442,0.730556,0.709580,0.730556
AdaBoostClassifier,0.690439,0.682649,0.681939,0.691667,0.688885,0.691667
GaussianProcessClassifier,0.799217,0.627946,0.587528,0.675000,0.612606,0.675000
DecisionTreeClassifier,0.623941,0.622113,0.620596,0.630556,0.628084,0.630556
GaussianNB,0.608815,0.568393,0.545137,0.602778,0.565224,0.602778
KNeighborsClassifier,0.589018,0.556905,0.538614,0.580556,0.554849,0.580556


In [32]:
burchard_by_london_VS_london_models_experiment.run_greed_search_cv_for_n_best_models(3)

running: RandomForestClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.7        0.73333333 0.75       0.75833333 0.75555556 0.74722222
 0.75277778 0.74722222 0.7        0.73333333 0.75       0.75833333
 0.75555556 0.74722222 0.75277778 0.74722222 0.56666667 0.56666667
 0.56388889 0.56388889 0.56388889 0.56388889 0.56388889 0.56388889
 0.72777778 0.75277778 0.73611111 0.75555556 0.75833333 0.76111111
 0.76111111 0.75833333 0.72777778 0.75277778 0.73611111 0.75555556
 0.75833333 0.76111111 0.76111111 0.75833333 0.57777778 0.57777778
 0.57222222 0.56944444 0.56666667 0.56388889 0.56388889 0.56388889
 0.73333333 0.76388889 0.75       0.76388889 0.76944444 0.775
 0.76944444 0.75833333 0.73333333 0.76388889 0.75       0.76388889
 0.76944444 0.775      0.76944444 0.75833333 0.57777778 0.58611111
 0.575      0.575      0.56944444 0.57222222 0.56944444 0.56388889
 0.70555556 0.73333333 0.74444444 0.7

Elapsed time to compute the greed search cv: 12.648 minutes
running: AdaBoostClassifier
Elapsed time to compute the greed search cv: 57.761 minutes
running: XGBClassifier
Elapsed time to compute the greed search cv: 62.064 minutes


In [33]:
for i in burchard_by_london_VS_london_models_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

0.7777777777777778
RandomForestClassifier(max_depth=11, n_estimators=300, random_state=0)
0.75
AdaBoostClassifier(learning_rate=0.5, n_estimators=300)
0.763888888888889
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.1, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=9, max_leaves=0, min_child_weight=3,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [34]:
burchard_by_zwickau_VS_london_models_experiment = thesisModelFeatures.ModelsExperiment(burchard_by_zwickau_VS_london_features_df)

In [35]:
burchard_by_zwickau_VS_london_models_experiment.run_cross_validate()

running: DecisionTreeClassifier
Elapsed time to compute the cross validate: 0.139 minutes
running: GaussianProcessClassifier
Elapsed time to compute the cross validate: 0.968 minutes
running: RandomForestClassifier
Elapsed time to compute the cross validate: 0.122 minutes
running: GaussianNB
Elapsed time to compute the cross validate: 0.072 minutes
running: KNeighborsClassifier
Elapsed time to compute the cross validate: 0.076 minutes
running: AdaBoostClassifier
Elapsed time to compute the cross validate: 0.712 minutes
running: XGBClassifier
Elapsed time to compute the cross validate: 2.086 minutes


In [36]:
burchard_by_zwickau_VS_london_models_experiment.show_cross_validate_results()

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
RandomForestClassifier,0.779559,0.735060,0.733672,0.758333,0.744240,0.758333
XGBClassifier,0.741122,0.724732,0.724051,0.738889,0.732281,0.738889
AdaBoostClassifier,0.728461,0.723244,0.722036,0.727778,0.726689,0.727778
GaussianProcessClassifier,0.799119,0.627708,0.587600,0.675000,0.612687,0.675000
DecisionTreeClassifier,0.625776,0.623125,0.620956,0.633333,0.629611,0.633333
GaussianNB,0.602903,0.575506,0.551375,0.611111,0.572094,0.611111
KNeighborsClassifier,0.589629,0.555446,0.535763,0.577778,0.551626,0.577778


In [37]:
burchard_by_zwickau_VS_london_models_experiment.run_greed_search_cv_for_n_best_models(3)

running: RandomForestClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.70555556 0.74166667 0.73888889 0.74444444 0.73888889 0.73888889
 0.75       0.74722222 0.70555556 0.74166667 0.73888889 0.74444444
 0.73888889 0.73888889 0.75       0.74722222 0.58888889 0.57222222
 0.56944444 0.56388889 0.56388889 0.56388889 0.56388889 0.56388889
 0.69444444 0.725      0.72777778 0.74444444 0.73611111 0.74444444
 0.75555556 0.75277778 0.69444444 0.725      0.72777778 0.74444444
 0.73611111 0.74444444 0.75555556 0.75277778 0.60833333 0.58611111
 0.58055556 0.56944444 0.56388889 0.56666667 0.56388889 0.56666667
 0.7        0.73333333 0.74444444 0.74722222 0.75       0.75
 0.76111111 0.75833333 0.7        0.73333333 0.74444444 0.74722222
 0.75       0.75       0.76111111 0.75833333 0.61388889 0.60833333
 0.59444444 0.58055556 0.575      0.58055556 0.57222222 0.56666667
 0.71666667 0.74166667 0.74444444 0.76

Elapsed time to compute the greed search cv: 12.516 minutes
running: AdaBoostClassifier
Elapsed time to compute the greed search cv: 57.366 minutes
running: XGBClassifier
Elapsed time to compute the greed search cv: 62.370 minutes


In [38]:
for i in burchard_by_zwickau_VS_london_models_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

0.7750000000000001
RandomForestClassifier(max_depth=9, n_estimators=500, random_state=0)
0.775
AdaBoostClassifier(learning_rate=0.5, n_estimators=300)
0.763888888888889
XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.1, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)


In [ ]:
# burchard_by_london_VS_zwickau_models_experiment.get_n_best_models(3)

In [ ]:
t= burchard_by_london_VS_zwickau_models_experiment.show_cross_validate_results().index[:3]

In [ ]:
t

In [ ]:
best_models_names = burchard_by_london_VS_zwickau_models_experiment.show_cross_validate_results().index[:3]
best_models = []

for m, name, greed_search_cv_params in burchard_by_london_VS_zwickau_models_experiment.models:
    if name in best_models_names: best_models.append(m)
best_models

In [ ]:
for m, classifier_name, greed_search_cv_params in best_models:
    print(classifier_name)

In [ ]:
for i in burchard_by_london_VS_zwickau_models_experiment.greed_search_cv_results:
    print(i.best_score_)
    print(i.best_estimator_)

In [ ]:
burchard_by_london_VS_zwickau_models_results = thesisModelFeatures.run_models(
    burchard_by_london_VS_zwickau_features_df
)

In [ ]:
burchard_by_london_VS_zwickau_models_results[0].sort_values(by=['accuracy'], ascending=False)

In [ ]:
'XGBClassifiers' in burchard_by_london_VS_zwickau_models_results[0].sort_values(by=['accuracy'], ascending=False).index[:3]

In [ ]:
burchard_by_zwickau_VS_zwickau_models_results = thesisModelFeatures.run_models(
    burchard_by_zwickau_VS_zwickau_features_df
)

In [ ]:
burchard_by_zwickau_VS_zwickau_models_results[0].sort_values(by=['accuracy'], ascending=False)

## burchard VS london

In [ ]:
burchard_by_london_VS_london_models_results = thesisModelFeatures.run_models(
    burchard_by_london_VS_london_features_df
)

In [ ]:
burchard_by_london_VS_london_models_results[0].sort_values(by=['accuracy'], ascending=False)

In [ ]:
burchard_by_zwickau_VS_london_models_results = thesisModelFeatures.run_models(
    burchard_by_zwickau_VS_london_features_df
)

In [ ]:
burchard_by_zwickau_VS_london_models_results[0].sort_values(by=['accuracy'], ascending=False)

## burchard VS zwickau greed search CV

In [ ]:
# burchard_by_london_VS_zwickau_greed_rearch_results = []
# for cls in [
#     'KNeighborsClassifier', 
#     'AdaBoostClassifier', 
#     'XGBClassifier'
# ]:
#     grid_search_cv_result = thesisModelFeatures.run_grid_search_cv(burchard_by_london_VS_zwickau_features_df, [cls])
#     burchard_by_london_VS_zwickau_greed_rearch_results.append([cls, grid_search_cv_result[1][0].best_score_, grid_search_cv_result[1][0].best_estimator_])

In [ ]:
# burchard_by_london_VS_zwickau_greed_rearch_results

In [ ]:
# burchard_by_zwickau_VS_zwickau_greed_rearch_results = []
# for cls in [
#     'KNeighborsClassifier', 
#     'AdaBoostClassifier', 
#     'XGBClassifier'
# ]:
#     grid_search_cv_result = thesisModelFeatures.run_grid_search_cv(burchard_by_zwickau_VS_zwickau_features_df, [cls])
#     burchard_by_zwickau_VS_zwickau_greed_rearch_results.append([cls, grid_search_cv_result[1][0].best_score_, grid_search_cv_result[1][0].best_estimator_])

In [ ]:
# burchard_by_zwickau_VS_zwickau_greed_rearch_results

## burchard VS london greed search CV

In [ ]:
# burchard_by_london_VS_london_greed_rearch_results = []
# for cls in [
#     'KNeighborsClassifier', 
#     'AdaBoostClassifier', 
#     'XGBClassifier'
# ]:
#     grid_search_cv_result = thesisModelFeatures.run_grid_search_cv(burchard_by_london_VS_london_features_df, [cls])
#     burchard_by_london_VS_london_greed_rearch_results.append([cls, grid_search_cv_result[1][0].best_score_, grid_search_cv_result[1][0].best_estimator_])

In [ ]:
# burchard_by_london_VS_london_greed_rearch_results

In [ ]:
# burchard_by_zwickau_VS_london_greed_rearch_results = []
# for cls in [
#     'KNeighborsClassifier', 
#     'AdaBoostClassifier', 
#     'XGBClassifier'
# ]:
#     grid_search_cv_result = thesisModelFeatures.run_grid_search_cv(burchard_by_zwickau_VS_london_features_df, [cls])
#     burchard_by_zwickau_VS_london_greed_rearch_results.append([cls, grid_search_cv_result[1][0].best_score_, grid_search_cv_result[1][0].best_estimator_])

## testing

In [ ]:
m = thesisModelFeatures.Model(xgb.XGBClassifier())

In [ ]:
m2 = thesisModelFeatures.Model(AdaBoostClassifier(learning_rate=1, n_estimators=2000))